In [1]:
import numpy as np

opcodeInput=np.loadtxt("input_day23.txt",delimiter=",",dtype=np.int64)

class OpCodeManager:
    def __init__(self, opcode):
        self.opcode = opcode
        self.relativeBaseIdx=0
    def __getitem__(self, idx):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        return self.opcode[idx]
    def __setitem__(self, idx, value):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        self.opcode[idx] = value

def decodeCommand(command):
    code=command%100
    p1Mode=int(command/100)%10
    p2Mode=int(command/1000)%10
    p3Mode=int(command/10000)%10
    return [code, p1Mode, p2Mode, p3Mode]

def getArgument(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return opcode[parameter]
    elif mode == 1:
        # immediate mode
        return parameter
    else:
        # relative mode
        return opcode[opcode.relativeBaseIdx+parameter]
    
def getWritePosition(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return parameter
    elif mode == 1:
        # immediate mode
        print("ERROR! Immediate mode not intended for writing")
        print(mode, parameter, opcode)
    else:
        # relative mode
        return opcode.relativeBaseIdx+parameter

In [2]:
import threading, queue

class Computer(threading.Thread):
    def __init__(self, opcodeMgr, queues, address,inputQueue,nat):
        super(Computer, self).__init__()
        self.opcode = opcodeMgr
        self.queues = queues
        self.address = address
        self.outputQueue=[]
        self.inputQueue=inputQueue
        self.nat=nat
        
    def sendOutput(self):
        recvAddr=self.outputQueue[0]
        X=self.outputQueue[1]
        Y=self.outputQueue[2]
        if recvAddr== 255:
            print("Addr ", self.address, " sending to NAT of packet ",( X, Y))
            self.nat.insert(0,(X,Y))
        else:
            self.queues[recvAddr].put_nowait((X,Y))
        self.outputQueue=[]
        
    def join(self, timeout=None):
        super(Computer, self).join(timeout)
        
    def isIdle(self):
        if self.queues[self.address].empty() and len(self.inputQueue) == 0 and len(self.outputQueue) == 0:
            return True
        return False
    
    def run(self):
        idx=0
    
        while True:

            decoded = decodeCommand(self.opcode[idx])
            if decoded[0] == 99:
                print("Finished!")
                return 0
            elif decoded[0] == 1:
                #add p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 + arg2
                idx+=4
            elif decoded[0] == 2:
                #multiply p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 * arg2
                idx+=4
            elif decoded[0] == 3:
                #input p1
                p1 = self.opcode[idx+1]
                arg1 = getWritePosition(self.opcode,decoded[1],p1)
                var=-1
                if len(self.inputQueue) > 0:
                    var = self.inputQueue.pop(0)
                else:
                    try:
                        inputPair = self.queues[self.address].get(False) # non-blocking wait, will throw exception if empty
                        var=inputPair[0]
                        self.inputQueue.append(inputPair[1])
                    except queue.Empty:
                        var=-1 # queue was empty
                #if var != -1:
                #    print("Input on addr ", self.address, ": ", var)
                self.opcode[arg1] = np.int64(var)
                idx+=2
            elif decoded[0] == 4:
                #output p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                self.outputQueue.append(arg1)
                #print("Output on addr ", self.address, ": ", arg1)
                if len(self.outputQueue) == 3:
                    self.sendOutput()
                idx+=2
            elif decoded[0] == 5:
                #jump-if-true p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 != 0:
                    idx=arg2
                else:      
                    idx=idx+3
            elif decoded[0] == 6:
                #jump-if-false p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 == 0:
                    idx=arg2
                else:      
                    idx+=3
            elif decoded[0] == 7:
                #less-than p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 < arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 8:
                #equals p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 == arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 9:
                # change relative base p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                self.opcode.relativeBaseIdx += arg1
                idx+=2
            else:
                print("Invalid code ", self.opcode, idx, self.opcode[idx], decoded)
                return

In [ ]:
# task 1
numAddresses=50
queues = [queue.Queue() for i in range(0,numAddresses)]

# Create the "thread pool"
pool = [Computer(opcodeMgr=OpCodeManager(opcodeInput.copy()), queues=queues,address=i,inputQueue=[i]) for i in range(0,numAddresses)]

# Start all threads
for thread in pool:
    thread.start()

# Ask threads to die and wait for them to do it
for thread in pool:
    thread.join()

Input on addr  0 :  0
Output on addr  0 :  24
Output on addr  0 :  100153
Output on addr  0 :  18508
Output on addr  0 :  42
Input on addr  1 :  1Output on addr  0 :  41179
Output on addr  0 :  18508
Output on addr  0 :  42
Output on addr  0 :  123537
Output on addr  0 :  18508
Output on addr  0 :  48
Output on addr  0 :  24151
Output on addr  0 :  18508
Output on addr  0 :  Input on addr  2
 :  2
Input on addr  3 :  3
48
Output on addr  0 :  48302
Output on addr  0 :  18508
Output on addr  0 :  48
Output on addr  0 :  72453
Input on addr  4 :  4
Output on addr  0 :  18508
Input on addr  5 :  5
Input on addr  6 :  6
Input on addr  7 :  7
Input on addr  8 :  8
Input on addr  9 :  9
Input on addr  10 :  10
Output on addr  10 :  27
Output on addr  10 :  188966
Output on addr  10 :  -62
Output on addr  9 :  1
Output on addr  9 :  68314
Output on addr  9 :  125
Input on addr  11 :  11
Output on addr  11Input on addr  1 :  Input on addr  12 :  12
Output on addr  12  :  68314
Input on addr  1

Input on addr  18 :  7369
Input on addr  18 :  1965899857408
Output on addr  18 :  43
Output on addr  18 :  60257
Output on addr  18 :  16685160936000
Input on addr  43 :  60257
Input on addr  43 :  16685160936000
Output on addr  43 :  255
Output on addr  43 :  26161
Output on addr  43 :  16685
Sending to address  255  requested of packet  (26161, 16685)


In [ ]:
# task 2
numAddresses=50
queues = [queue.Queue() for i in range(0,numAddresses)]

natMemory=[]

# Create the "thread pool"
pool = [Computer(opcodeMgr=OpCodeManager(opcodeInput.copy()), queues=queues,address=i,inputQueue=[i],nat=natMemory) for i in range(0,numAddresses)]

# Start all threads
for thread in pool:
    thread.start()

formerY=-1
while True:
    # check if IDLE
    idle=True
    for thread in pool:
        idle = idle and thread.isIdle()
    if idle and len(natMemory) > 0:
        print("NAT: sending ", natMemory[0], " to 0")
        if formerY==natMemory[0][1]:
            print("SOLUTION: ", formerY)
        formerY=natMemory[0][1]
        queues[0].put_nowait(natMemory[0])
    
# Ask threads to die and wait for them to do it
for thread in pool:
    thread.join()

Addr  43  sending to NAT of packet  (26161, 16685)
NAT: sending  (26161, 16685)  to 0
Addr  43  sending to NAT of packet  (26161, 17803)
Addr  43  sending to NAT of packet  (26161, 18811)
Addr  43  sending to NAT of packet  (26161, 18187)
Addr  43  sending to NAT of packet  (26161, 17624)
Addr  43  sending to NAT of packet  (26161, 17116)
Addr  43  sending to NAT of packet  (26161, 15173)
NAT: sending  (26161, 15173)  to 0
Addr  43  sending to NAT of packet  (26161, 13560)
Addr  43  sending to NAT of packet  (26161, 13140)
Addr  43  sending to NAT of packet  (26161, 12757)
Addr  43  sending to NAT of packet  (26161, 12409)
Addr  43  sending to NAT of packet  (26161, 13245)
Addr  43  sending to NAT of packet  (26161, 14005)
NAT: sending  (26161, 14005)  to 0
Addr  43  sending to NAT of packet  (26161, 13736)
Addr  43  sending to NAT of packet  (26161, 13488)
Addr  43  sending to NAT of packet  (26161, 13259)
Addr  43  sending to NAT of packet  (26161, 12014)
Addr  43  sending to NAT of 